## PREPARATION

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

# Download NLTK resources (jika belum ada)
nltk.download('punkt')
nltk.download('stopwords')
# Download the missing 'punkt_tab' resource
nltk.download('punkt_tab')

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Inisialisasi daftar stopwords Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Load dataset
try:
    df = pd.read_csv('GojekAppReview_1.csv')
except FileNotFoundError:
    print("File 'GojekAppReview_1.csv' tidak ditemukan. Pastikan file sudah diunggah dengan benar.")

print("Contoh Dataset:")
print(df.head())
print(f"\nJumlah baris data: {len(df)}")

Contoh Dataset:
                  userName                                            content  \
0                Yuga Edit                            akun gopay saya di blok   
1                 ff burik  Lambat sekali sekarang ini bosssku apk gojek g...   
2  Anisa Suci Rahmayuliani  Kenapa sih dari kemarin sy buka aplikasi gojek...   
3             naoki yakuza  Baru download gojek dan hape baru trus ditop u...   
4            Trio Sugianto                                             Mantap   

   score                   at appVersion  
0      1  2022-01-21 10:52:12      4.9.3  
1      3  2021-11-30 15:40:38      4.9.3  
2      4  2021-11-29 22:58:12      4.9.3  
3      1  2022-09-03 15:21:17      4.9.3  
4      5  2022-01-15 10:05:27      4.9.3  

Jumlah baris data: 225002


## Preprocessing Data

Labelling

In [ ]:
def label_sentiment(score):
    if score in [1, 2]:
        return 'negatif'
    elif score == 3:
        return 'netral'
    elif score in [4, 5]:
        return 'positif'
    return None # Untuk skor yang mungkin tidak valid

df['sentiment'] = df['score'].apply(label_sentiment)

# Hapus baris dengan sentimen yang tidak terdefinisi (jika ada)
df.dropna(subset=['sentiment'], inplace=True)

print("\nDataset setelah labelling:")
print(df[['score', 'sentiment']].head())
print(f"\nDistribusi Sentimen:\n{df['sentiment'].value_counts()}")


Dataset setelah labelling:
   score sentiment
0      1   negatif
1      3    netral
2      4   positif
3      1   negatif
4      5   positif

Distribusi Sentimen:
sentiment
positif    161371
negatif     54171
netral       9460
Name: count, dtype: int64


Cleaning Text

In [ ]:
def clean_text(text):
    if isinstance(text, str):
        text = text.lower() # Lowercasing
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Hapus URL
        text = re.sub(r'\@\w+|\#','', text) # Hapus mention dan hashtag
        text = re.sub(r'[^\w\s]', '', text) # Hapus karakter khusus dan tanda baca
        text = re.sub(r'\d+', '', text) # Hapus angka
        # Hapus emoji (basic)
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)
        text = text.strip() # Hapus spasi berlebih di awal dan akhir
    return text

df['cleaned_content'] = df['content'].apply(clean_text)
print("\nDataset setelah pembersihan teks:")
print(df[['content', 'cleaned_content']].head())


Dataset setelah pembersihan teks:
                                             content  \
0                            akun gopay saya di blok   
1  Lambat sekali sekarang ini bosssku apk gojek g...   
2  Kenapa sih dari kemarin sy buka aplikasi gojek...   
3  Baru download gojek dan hape baru trus ditop u...   
4                                             Mantap   

                                     cleaned_content  
0                            akun gopay saya di blok  
1  lambat sekali sekarang ini bosssku apk gojek g...  
2  kenapa sih dari kemarin sy buka aplikasi gojek...  
3  baru download gojek dan hape baru trus ditop u...  
4                                             mantap  


Normalisasi Kata Tidak Baku

In [ ]:
normalization_dict = {
    'yg': 'yang', 'jg': 'juga', 'jga': 'juga', 'ga': 'tidak', 'gak': 'tidak', 'gk': 'tidak',
    'tdk': 'tidak', 'nggak': 'tidak', 'ngga': 'tidak', 'enggak': 'tidak',
    'utk': 'untuk', 'buat': 'untuk', 'dr': 'dari', 'dp': 'dapat', 'dg': 'dengan',
    'sy': 'saya', 'sya': 'saya', 'aku': 'saya', 'aq': 'saya', 'gua': 'saya', 'gw': 'saya',
    'lu': 'kamu', 'loe': 'kamu', 'lo': 'kamu', 'km': 'kamu', 'kmu': 'kamu',
    'apk': 'aplikasi', 'apknya': 'aplikasinya', 'app': 'aplikasi',
    'bosssku': 'bosku', 'boskuhh': 'bosku', 'bossku': 'bosku',
    'kaya': 'seperti', 'kyk': 'seperti', 'ky': 'seperti',
    'kalo': 'kalau', 'kl': 'kalau', 'klu': 'kalau',
    'udah': 'sudah', 'sdh': 'sudah', 'udh': 'sudah',
    'aja': 'saja', 'doang': 'saja', 'tok': 'saja',
    'blm': 'belum', 'bloom': 'belum',
    'bgt': 'banget', 'bngt': 'banget', 'bgtss': 'banget', 'bangettt': 'banget',
    'skrg': 'sekarang', 'skrng': 'sekarang',
    'kmrn': 'kemarin', 'kmren': 'kemarin',
    'besok2': 'besok-besok',
    'bsk': 'besok',
    'trs': 'terus', 'trus': 'terus',
    'lg': 'lagi', 'lgi': 'lagi',
    'mau': 'ingin', 'mo': 'mau',
    'dpt': 'dapat', 'dapet': 'dapat',
    'ntr': 'nanti', 'ntar': 'nanti',
    'krn': 'karena', 'karna': 'karena',
    'pdhl': 'padahal',
    'org': 'orang',
    'y' : 'ya',
    'jd': 'jadi',
    'ajaib': 'ajaib',  # biarkan karena bisa nama brand juga
    'ni': 'ini', 'nih': 'ini',
    'sih': '', 'deh': '', 'dong': '', 'ya': '', 'kok': '', 'lah': '',
    'tp': 'tapi', 'tpn': 'tapi', 'tapii': 'tapi',
    'btw': 'omong-omong',
    'mantul': 'mantap betul', 'mantappp': 'mantap',
    'ok': 'oke', 'okehh': 'oke',
    'thx': 'terima kasih', 'makasih': 'terima kasih', 'mksh': 'terima kasih',
    'trims': 'terima kasih', 'terimakasih': 'terima kasih',
    'cpt': 'cepat', 'cepet': 'cepat', 'cepetan': 'cepat',
    'lemot': 'lambat',
    'parahh': 'parah', 'parahhh': 'parah',
    'loyalti': 'loyalty',  # bisa jadi nama fitur
    'driverny': 'drivernya', 'ojol': 'ojek online',
    'cs': 'customer service',
    'fiturnya': 'fitur',
    'layananx': 'layanan',
    'gaes': 'teman-teman', 'gengs': 'teman-teman',
    'cashbacknya': 'cashback',
    'promo2': 'promo-promo', 'promonya': 'promo',
    'lok': 'lokasi',
    'respon': 'respons',
    'baguss': 'bagus', 'bgs': 'bagus', 'baguuus': 'bagus',
    'burukk': 'buruk', 'jelekkk': 'jelek',
    'ngaret': 'terlambat',
    'delay': 'terlambat',
    'crash': 'rusak',
    'hang': 'macet',
    'eror': 'error', 'erorr': 'error', 'erorrnya': 'error',
    'kecewaaa': 'kecewa',
    'toppp': 'top',
    'makin': 'semakin',
    'bingitz': 'banget',
    'ribbed': 'ribet',
}


def normalize_text(text):
    if isinstance(text, str):
        words = text.split()
        normalized_words = [normalization_dict.get(word, word) for word in words]
        return ' '.join(normalized_words)
    return text

df['normalized_content'] = df['cleaned_content'].apply(normalize_text)
print("\nDataset setelah normalisasi kata:")
print(df[['cleaned_content', 'normalized_content']].head())


Dataset setelah normalisasi kata:
                                     cleaned_content  \
0                            akun gopay saya di blok   
1  lambat sekali sekarang ini bosssku apk gojek g...   
2  kenapa sih dari kemarin sy buka aplikasi gojek...   
3  baru download gojek dan hape baru trus ditop u...   
4                                             mantap   

                                  normalized_content  
0                            akun gopay saya di blok  
1  lambat sekali sekarang ini bosku aplikasi goje...  
2  kenapa  dari kemarin saya buka aplikasi gojek ...  
3  baru download gojek dan hape baru terus ditop ...  
4                                             mantap  


Tokenisasi dan Penghapusan Stopwords

In [ ]:
sentiment_keywords = {
    # Kata tanya yang menunjukkan keluhan atau masalah
    "kenapa", "mengapa", "bagaimana", "apa", "kok",

    # Kata kerja keluhan atau permintaan
    "perbaiki", "tolong", "hapus", "ganti", "perbarui", "hilangkan", "tambah", "kurangi",

    # Kata sifat bernuansa sentimen negatif
    "lemot", "lambat", "buruk", "jelek", "error", "macet", "susah", "ribet", "parah", "nge-bug", "crash", "hilang",

    # Kata sifat bernuansa sentimen positif
    "bagus", "mantap", "cepat", "mudah", "keren", "top", "oke", "hebat", "baik",

    # Kata ekspresif dan interjeksi yang bisa bernilai sentimen
    "yah", "duh", "astaga", "sayang", "wow", "alhamdulillah", "aduh", "please", "thanks", "terima", "syukur",

    # Kata berkaitan dengan fungsi aplikasi
    "login", "bayar", "pesan", "promo", "diskon", "voucher", "fitur", "layanan", "akses", "notifikasi"
}

In [ ]:
# Ubah stopword jadi set
stop_words = set(stop_words)

# Hapus kata-kata penting dari daftar stopwords
stop_words -= sentiment_keywords  # buang dari stopwords

In [ ]:
def tokenize_and_remove_stopwords(text):
    if isinstance(text, str):
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words and word.isalpha()] # Hapus stopwords dan token non-alfabet
        return tokens
    return []

df['tokenized_content'] = df['normalized_content'].apply(tokenize_and_remove_stopwords)
print("\nDataset setelah tokenisasi dan penghapusan stopwords:")
print(df[['normalized_content', 'tokenized_content']].head())


Dataset setelah tokenisasi dan penghapusan stopwords:
                                  normalized_content  \
0                            akun gopay saya di blok   
1  lambat sekali sekarang ini bosku aplikasi goje...   
2  kenapa  dari kemarin saya buka aplikasi gojek ...   
3  baru download gojek dan hape baru terus ditop ...   
4                                             mantap   

                                   tokenized_content  
0                                [akun, gopay, blok]  
1                   [lambat, bosku, aplikasi, gojek]  
2  [kenapa, kemarin, buka, aplikasi, gojek, kasih...  
3  [download, gojek, hape, ditop, u, gopay, trans...  
4                                           [mantap]  


Stemming/Lemmatization (Menggunakan Sastrawi)

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm

# Tqdm support for pandas apply
tqdm.pandas(desc="Stemming")

def stem_text(tokens):
    return [stemmer.stem(token) for token in tokens]

# Terapkan stemming dengan progress bar
df['processed_content'] = df['tokenized_content'].progress_apply(lambda tokens: ' '.join(stem_text(tokens)))

print("\nDataset setelah stemming:")
print(df[['tokenized_content', 'processed_content']].head())

# Hapus baris kosong setelah preprocessing
df = df[df['processed_content'].str.strip().astype(bool)]
print(f"\nJumlah baris data setelah preprocessing: {len(df)}")


Stemming: 100%|██████████| 225002/225002 [2:05:22<00:00, 29.91it/s]



Dataset setelah stemming:
                                   tokenized_content  \
0                                [akun, gopay, blok]   
1                   [lambat, bosku, aplikasi, gojek]   
2  [kenapa, kemarin, buka, aplikasi, gojek, kasih...   
3  [download, gojek, hape, ditop, u, gopay, trans...   
4                                           [mantap]   

                                   processed_content  
0                                    akun gopay blok  
1                          lambat bos aplikasi gojek  
2  kenapa kemarin buka aplikasi gojek kasih binta...  
3  download gojek hape top u gopay transaksi dial...  
4                                             mantap  

Jumlah baris data setelah preprocessing: 223855


In [ ]:
df.to_csv('processed_gojek_reviews.csv', index=False)

print("\nDataFrame hasil preprocessing telah disimpan di runtime.")



DataFrame hasil preprocessing telah disimpan di runtime.
